In [9]:
import pandas as pd
import requests
import urllib.parse

**Data Source**: [City of Chicago: Crimes 2001 to Present](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/data)



## Filtering


To be more effecient with network resources, Chicago exposes an API endpoint which allows us to filter the data and only download the data that we want.  It apparently uses a backend query language called SoQL, which gives us the ability to use SQL like query language to filter the data how we want.

https://dev.socrata.com/foundry/data.cityofchicago.org/ijzp-q8t2

In [3]:
def build_query_str(query: dict) -> str:
    """Parses a dictionary of parameters to build a SODA query string"""
    out = ''
    if query:
        for k, v in query.items():
            out += f"${k}={v}"
    return out

In [4]:
"""
Example of a functioning api call

TODO: Clean up this code structure
"""

endpoint = 'https://data.cityofchicago.org/resource/ijzp-q8t2.json'
query = {
    'where': """
        date > '2019'
        """,
}
soda_query = build_query_str(query)
response = requests.get(''.join([endpoint,'?',soda_query]))
temp = pd.DataFrame(response.json())
temp.head(3)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11552605,JC100030,2019-01-01T00:01:00.000,004XX N MONTICELLO AVE,143A,WEAPONS VIOLATION,UNLAWFUL POSS OF HANDGUN,ALLEY,True,False,...,27,23,15,1151958,1902815,2019,2019-01-10T15:16:50.000,41.889196391,-87.717403722,"{'latitude': '41.889196391', 'longitude': '-87..."
1,11552645,JC100087,2019-01-01T00:01:00.000,028XX E 76TH ST,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,7,43,14,1196262,1855497,2019,2019-01-10T15:16:50.000,41.75836602,-87.556276032,"{'latitude': '41.75836602', 'longitude': '-87...."
2,11552869,JC100416,2019-01-01T00:01:00.000,001XX N CLARK ST,0820,THEFT,$500 AND UNDER,STREET,False,False,...,42,32,06,1175524,1900930,2019,2019-01-10T15:16:50.000,41.883527057,-87.630917053,"{'latitude': '41.883527057', 'longitude': '-87..."


In [8]:
urllib.parse.urlencode(query, )

'where=%0A++++++++date+%3E+%272019%27%0A++++++++'

In [5]:
''.join([endpoint,'?',soda_query]) #What does the string look like?

"https://data.cityofchicago.org/resource/ijzp-q8t2.json?$where=\n        date > '2019'\n        "

In [6]:
temp.columns.to_list()

['id',
 'case_number',
 'date',
 'block',
 'iucr',
 'primary_type',
 'description',
 'location_description',
 'arrest',
 'domestic',
 'beat',
 'district',
 'ward',
 'community_area',
 'fbi_code',
 'x_coordinate',
 'y_coordinate',
 'year',
 'updated_on',
 'latitude',
 'longitude',
 'location']

## Paging

<div markdown="1" 
    style= "
    box-sizing: border-box;
    padding: 15px;
    margin-bottom: 20px;
    border: 1px solid transparent;
    border-radius: 15px;
    color: #31708f;
    background-color: #d9edf7;
    border-color: #bce8f1;
    width: 55%;
    ">

**Heads Up!**  
The order of the results of a query are not implicitly ordered, so if you're paging, make sure you provide an [`$order` clause](https://dev.socrata.com/docs/queries/) or at a minimum `$order=:id`.  
That will guarantee that the order of your results will be stable as you page through the dataset.
</div>

As is the usual case with API acquire methods, we are going to need to loop through several pages.  To help me with that, I want to first define a function that gets a single page and returns the JSON.